In [ ]:
# 对客户提供的关于的电池的数据进行预处理
# 创建一个EMR集群
# EMR version 5.33.0
# aws emr create-cluster --termination-protected --applications Name=Hadoop Name=JupyterEnterpriseGateway Name=Spark Name=Livy --ec2-attributes '{"KeyName":"key-*****-new","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"subnet-08e5c1c41981c****","EmrManagedSlaveSecurityGroup":"sg-0d5296d5ecc31****","EmrManagedMasterSecurityGroup":"sg-0f5a587aa2c9d****"}' --release-label emr-5.33.0 --log-uri 's3n://aws-logs-*****-cn-northwest-1/elasticmapreduce/' --instance-groups '[{"InstanceCount":2,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":32,"VolumeType":"gp2"},"VolumesPerInstance":2}]},"InstanceGroupType":"CORE","InstanceType":"m5.xlarge","Name":"Core - 2"},{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":32,"VolumeType":"gp2"},"VolumesPerInstance":2}]},"InstanceGroupType":"MASTER","InstanceType":"m5.xlarge","Name":"Master - 1"}]' --configurations '[{"Classification":"spark-hive-site","Properties":{"hive.metastore.client.factory.class":"com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory"}}]' --auto-scaling-role EMR_AutoScaling_DefaultRole --ebs-root-volume-size 30 --service-role EMR_DefaultRole --enable-debugging --name 'pysparktest4' --scale-down-behavior TERMINATE_AT_TASK_COMPLETION --region cn-northwest-1
# EMR version 6.3.0
# aws emr create-cluster --termination-protected --applications Name=Hadoop Name=JupyterHub Name=JupyterEnterpriseGateway Name=Spark Name=Livy --ec2-attributes '{"KeyName":"key-*****-new","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"subnet-08e5c1c41981****","EmrManagedSlaveSecurityGroup":"sg-0d5296d5ecc31****","EmrManagedMasterSecurityGroup":"sg-0f5a587aa2c9d****"}' --release-label emr-6.3.0 --log-uri 's3n://aws-logs-*****-cn-northwest-1/elasticmapreduce/' --instance-groups '[{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":32,"VolumeType":"gp2"},"VolumesPerInstance":2}]},"InstanceGroupType":"MASTER","InstanceType":"m5.xlarge","Name":"Master - 1"},{"InstanceCount":2,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":32,"VolumeType":"gp2"},"VolumesPerInstance":2}]},"InstanceGroupType":"CORE","InstanceType":"m5.xlarge","Name":"Core - 2"}]' --configurations '[{"Classification":"spark-hive-site","Properties":{"hive.metastore.client.factory.class":"com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory"}}]' --auto-scaling-role EMR_AutoScaling_DefaultRole --ebs-root-volume-size 25 --service-role EMR_DefaultRole --enable-debugging --name 'pysparktest5' --scale-down-behavior TERMINATE_AT_TASK_COMPLETION --region cn-northwest-1

In [ ]:
# 引入需要的包
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# 创建了spark session
spark = SparkSession.builder.getOrCreate()

In [ ]:
# 读取存放在s3上的原始数据，展现他们的表结构和其中头2行信息
# 多个CSV文件 s3:///*******/*******_20200402215601.CSV 和 s3:///*******//*******_20200402215609.CSV

df = spark.read.format('csv').options(header='true', inferSchema='true').option("encoding", "GBK").load("s3:///*******//*******_20200402215601.CSV")
df.printSchema()
df.head(1)
#df.count()

In [ ]:
# 合并多个CSV文件到单个文件
from pyspark.sql.functions import input_file_name
from pyspark.sql import SQLContext
#from pyspark.sql.types import *
sqlContext = SQLContext(sc)

fullpath_csv = 's3:///*******/*.CSV'
out_csv = 's3:///*******/merger/all.CSV'

df = spark.read.format("csv") \
   .options(header='true', inferSchema='true') \
   .option("encoding", "GBK") \
   .load(fullpath) 
#df.printSchema()
#df.head(1)
df.count()

df.write.option("header","true").csv(out_csv)

df = spark.read.format("csv") \
   .options(header='true', inferSchema='true') \
   .option("encoding", "GBK") \
   .load(out_csv) 
#df.printSchema()
#df.head(1)
df.count()

In [ ]:
#  定义英文的schema
#  车架号: string       vin",StringType,true)                      
#  车牌号: string       carnum",StringType,true)                      
#  IMEI: decimal        IMEI",StringType,true)              
#  采集时间: string       collecttime",TimestampType,true)       
#  车辆状态: string       carstatus",StringType,true)       
#  充电状态: string       chargestatus",StringType,true)       
#  运行模式: string       runmoble",StringType,true)       
#  车速(km/h): string       speed",StringType,true)           
#  累计里程(km): double       odometer",StringType,true)           
#  总电压(V): string       totalvol",DoubleType,true)    *            
#  总电流(A): string       totalcurr",DoubleType,true)   *            
#  SOC(%): string       soc",StringType,true)         
#  DC-DC状态: string       dcstatus",StringType,true)          
#  有驱动力: string       withdriving",StringType,true)       
#  有制动力: string       withbreaking",StringType,true)       
#  档位: string       runlevel",StringType,true)     
#  绝缘电阻(kΩ): string       resistance",StringType,true)           
#  加速踏板行程值: string       accelevalue",StringType,true)          
#  制动踏板状态: string       breakstatus",StringType,true)         
#  驱动电机个数: integer       nummotor",IntegerType,true)          
#  驱动电机序号: string       snmotor",StringType,true)         
#  驱动电机状态: string       motorstatus",StringType,true)         
#  驱动电机控制器温度(℃): string      motorcoltemp",StringType,true)               
#  驱动电机转速(r/min): string      motordrispeed",StringType,true)                
#  驱动电机转矩(N*m): string       motordritorque",StringType,true)              
#  驱动电机温度(℃): string       motortemp",StringType,true)            
#  电机控制器输入电压(V): string      motcolinputvol",StringType,true)               
#  直流母线电流(A): string       dcbusbarcurr",StringType,true)            
#  发动机状态: string       enginestatus",StringType,true)        
#  曲轴转速(rpm): string       crshaftspeed",StringType,true)            
#  燃料消耗率(L/100km): string      fuelconrate",StringType,true)                 
#  定位状态: string       gpsstatus",StringType,true)       
#  纬度状态: string       latstatus",StringType,true)       
#  经度状态: string       longstatus",StringType,true)       
#  经度: double       long",DoubleType,true)     
#  纬度: double       lat",DoubleType,true)     
#  最高电压电池子系统号: integer      maxvolbatsubnum",StringType,true)              
#  最高电压电池单体代号: string      maxvolbatunitnum",StringType,true)             
#  电池单体电压最高值(V): string      maxvoltpercell",StringType,true)   *                
#  最低电压电池子系统号: integer      lowvolbatsubnum",StringType,true)                
#  最低电压电池单体代号: string      lowvolbatunitnum",StringType,true)             
#  电池单体电压最低值(V): string      lowvoltpercell",StringType,true)    *                 
#  最高温度子系统号: integer       maxtempsubnum",StringType,true)            
#  最高温度探针单体代号: string      maxtemprobecode",StringType,true)             
#  最高温度值(℃): string       maxtemperature",StringType,true)      *            
#  最低温度子系统号: integer       mintemsubsnum",StringType,true)            
#  最低温度探针子系统代号: string      mintemprobecode",StringType,true)              
#  最低温度值(℃): string       lowtemperature",StringType,true)      *              
#  最高报警等级: string       upalarmlevel",StringType,true)         
#  可充电储能装置故障总数: interger      numfailrecharenergy",StringType,true)               
#  驱动电机故障总数: integer       nummotorfail",IntegerType,true)            
#  发动机故障总数: integer       numenginefail",IntegerType,true)           
#  其它故障总数: integer       numotherfail",IntegerType,true)          
#  可充电储能装置故障代码列表:      codelistrechardev",StringType,true)                
#  驱动电机故障代码列表: string      codelistmotfaul",StringType,true)             
#  发动机故障列表: string       codelistengine",StringType,true)          
#  其他故障代码列表: string       codelistother",StringType,true)           
#  温度差异报警: string       tempdifalarm",StringType,true)         
#  电池高温报警: string       bathightemalarm",StringType,true)         
#  车载储能装置类型过压报警: string      vehenergydevovervolalarm",StringType,true)               
#  车载储能装置类型欠压报警: string     vehenergydevlowvolalarm",StringType,true)               
#  SOC低报警: string       lowsocalarm",StringType,true)         
#  单体电池过压报警: string       cellovervoltalarm",StringType,true)           
#  单体电池欠压报警: string       celllowvoltalarm",StringType,true)           
#  SOC过高报警: string       oversocalarm",StringType,true)          
#  SOC跳变报警: string       jumpsocalarm",StringType,true)          
#  可充电储能系统不匹配报警: string      recharenergymisalarm",StringType,true)               
#  电池单体一致性差报警: string      cellpooralarm",StringType,true)    *             
#  绝缘报警: string       insulatalarm",StringType,true)       
#  DC-DC温度报警: string       dctempalarm",StringType,true)            
#  制动系统报警: string       breaksysalarm",StringType,true)         
#  DC-DC状态报警: string       dcstatualarm",StringType,true)            
#  驱动电机控制器温度报警: string      motorcontemalarm",StringType,true)              
#  高压互锁状态报警: string       highvolinterlockalarm",StringType,true)           
#  驱动电机温度报警: string       mottemalarm",StringType,true)           
#  车载储能装置类型过充: string      energydevovercharge",StringType,true)             
#  电压可充电储能子系统个数: interger      numvolrechargsub",IntegerType,true)                
#  可充电储能装置电压: string       rechargedevvol",StringType,true)            
#  可充电储能装置电流: string       rechargedevcurr",StringType,true)            
#  单体电池总数: string       cellnum",StringType,true)         
#  电机信息: string       motorinfo",StringType,true)       
#  温度可充电储能子系统个数: interger      numtemprecharsub",IntegerType,true)                
#  温度探针总数: string       numbtemprob",StringType,true)         
#  电池包电压信息: string       batteryvolinfo",StringType,true)          
#  可充电储能装置温度数据: string      rechargedevtempinfo",StringType,true)              
#  可充电储能装置电压数据: string      rechargedevvolinfo",StringType,true)              
#  电池包温度信息: string       batterytempinfo",StringType,true)          
#  可充电储能子系统号: string       rechargesubsn",StringType,true)   

data_schema = [StructField('vin', StringType(), True),
               StructField('carnum', StringType(), True),
               StructField('IMEI', StringType(), True),
               StructField('collecttime', TimestampType(), True),
               StructField('carstatus', StringType(), True),
               StructField('chargestatus', StringType(), True),
               StructField('runmoble', StringType(), True),
               StructField('speed', StringType(), True),
               StructField('odometer', StringType(), True),
               StructField('totalvol', DoubleType(), True),
               StructField('totalcurr', DoubleType(), True),
               StructField('soc', StringType(), True),
               StructField('dcstatus', StringType(), True),
               StructField('withdriving', StringType(), True),
               StructField('withbreaking', StringType(), True),
               StructField('runlevel', StringType(), True),
               StructField('resistance', StringType(), True),
               StructField('accelevalue', StringType(), True),
               StructField('breakstatus', StringType(), True),
               StructField('nummotor', IntegerType(), True),
               StructField('snmotor', StringType(), True),
               StructField('motorstatus', StringType(), True),
               StructField('motorcoltemp', StringType(), True),
               StructField('motordrispeed', StringType(), True),
               StructField('motordritorque', StringType(), True),
               StructField('motortemp', StringType(), True),
               StructField('motcolinputvol', StringType(), True),
               StructField('dcbusbarcurr', StringType(), True),
               StructField('enginestatus', StringType(), True),
               StructField('crshaftspeed', StringType(), True),
               StructField('fuelconrate', StringType(), True),
               StructField('gpsstatus', StringType(), True),
               StructField('latstatus', StringType(), True),
               StructField('longstatus', StringType(), True),
               StructField('long', DoubleType(), True),
               StructField('lat', DoubleType(), True),
               StructField('maxvolbatsubnum', StringType(), True),
               StructField('maxvolbatunitnum', StringType(), True),
               StructField('maxvoltpercell', StringType(), True),
               StructField('lowvolbatsubnum', StringType(), True),
               StructField('lowvolbatunitnum', StringType(), True),
               StructField('lowvoltpercell', StringType(), True),
               StructField('maxtempsubnum', StringType(), True),
               StructField('maxtemprobecode', StringType(), True),
               StructField('maxtemperature', StringType(), True),
               StructField('mintemsubsnum', StringType(), True),
               StructField('mintemprobecode', StringType(), True),
               StructField('lowtemperature', StringType(), True),
               StructField('upalarmlevel', StringType(), True),
               StructField('numfailrecharenergy', StringType(), True),
               StructField('nummotorfail', IntegerType(), True),
               StructField('numenginefail', IntegerType(), True),
               StructField('numotherfail', IntegerType(), True),
               StructField('codelistrechardev', StringType(), True),
               StructField('codelistmotfaul', StringType(), True),
               StructField('codelistengine', StringType(), True),
               StructField('codelistother', StringType(), True),
               StructField('tempdifalarm', StringType(), True),
               StructField('bathightemalarm', StringType(), True),
               StructField('vehenergydevovervolalarm', StringType(), True),
               StructField('vehenergydevlowvolalarm', StringType(), True),
               StructField('lowsocalarm', StringType(), True),
               StructField('cellovervoltalarm', StringType(), True),
               StructField('celllowvoltalarm', StringType(), True),
               StructField('oversocalarm', StringType(), True),
               StructField('jumpsocalarm', StringType(), True),
               StructField('recharenergymisalarm', StringType(), True),
               StructField('cellpooralarm', StringType(), True),
               StructField('insulatalarm', StringType(), True),
               StructField('dctempalarm', StringType(), True),
               StructField('breaksysalarm', StringType(), True),
               StructField('dcstatualarm', StringType(), True),
               StructField('motorcontemalarm', StringType(), True),
               StructField('highvolinterlockalarm', StringType(), True),
               StructField('mottemalarm', StringType(), True),
               StructField('energydevovercharge', StringType(), True),
               StructField('numvolrechargsub', IntegerType(), True),
               StructField('rechargedevvol', StringType(), True),
               StructField('rechargedevcurr', StringType(), True),
               StructField('cellnum', StringType(), True),
               StructField('motorinfo', StringType(), True),
               StructField('numtemprecharsub', IntegerType(), True),
               StructField('numbtemprob', StringType(), True),
               StructField('batteryvolinfo', StringType(), True),
               StructField('rechargedevtempinfo', StringType(), True),
               StructField('rechargedevvolinfo', StringType(), True),
               StructField('batterytempinfo', StringType(), True),
               StructField('rechargesubsn', StringType(), True)]

schema = StructType(fields=data_schema)

out_csv = 's3:///*******/merger/all.CSV'

df = spark.read.format("csv") \
   .options(header='true') \
   .schema(schema) \
   .option("encoding", "UTF8") \
   .option("charset", "UTF-8") \
   .load(out_csv) 

#df.printSchema()
# 利用PySpark DataFrame的内置函数查询，和删除空值行
#df.select( "totalvol","totalcurr","maxvoltpercell","lowvoltpercell","maxtemperature","lowtemperature","cellpooralarm").show(30)
df = df.filter(df.totalvol.isNotNull())
df.count()

In [ ]:
# 注册dataframe 为一个临时的view
# 利用标准的SQL语句查询，查询英文schema的结果
df.createOrReplaceTempView("battery")

# 查询异常数据, 空值删除
query = 'select count(*) from battery where totalvol is null'
sql_results = spark.sql(query)
sql_results.show()

query = 'select * from battery where totalvol is null'
sql_results = spark.sql(query)
sql_results.show()

In [ ]:
# 将处理后的数据保存到S3
final_csv = 's3:///*******/final/final.CSV'
df_final = df.select( "totalvol","totalcurr","maxvoltpercell","lowvoltpercell","maxtemperature","lowtemperature","cellpooralarm")

df_final.write.option("header","true").csv(final_csv)

In [ ]:
# 检查最后保存的数据
df_check = spark.read.format("csv") \
           .options(header='true', inferSchema='true') \
           .option("encoding", "GBK") \
           .load(final_csv) 
df_check.printSchema()
df_check.head(1)
df_check.count()